In [29]:
import pandas as pd
from itertools import combinations
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [30]:
transactions = [
    ['I1', 'I2', 'I5'],
    ['I2', 'I4'],
    ['I2', 'I3'],
    ['I1', 'I2', 'I4'],
    ['I1', 'I3'],
    ['I2', 'I3'],
    ['I1', 'I3'],
    ['I1', 'I2', 'I3', 'I5'],
    ['I1', 'I2', 'I3']
]

# Implementação explícita

**Algoritmo Apriori**

**Entrada:**
- D: base de dados de transações
- supMin: suporte mínimo

**Saída:**
- Todos os conjuntos frequentes com suporte ≥ supMin

---

1. F₁ ← conjuntos frequentes de tamanho 1 (com suporte ≥ supMin)
2. k ← 1

3. enquanto Fₖ ≠ ∅ faça:
   4. k ← k + 1
   5. Cₖ ← gerar todos os candidatos de tamanho k a partir de Fₖ₋₁
   6. para cada transação t ∈ D faça:
      7. para cada candidato c ∈ Cₖ:
         se c ⊆ t, então incrementar contador de suporte de c
   8. Fₖ ← {c ∈ Cₖ | suporte(c) ≥ supMin}

9. fim-enquanto

10. Resultado ← ⋃ Fₖ (união de todos os conjuntos frequentes encontrados)


In [31]:
supMin = 2
n_trans = len(transactions)

# Etapa 1: contar itens únicos
item_counts = {}
for t in transactions:
    for item in t:
        key = tuple([item])
        item_counts[key] = item_counts.get(key, 0) + 1

# F1: itemsets frequentes de tamanho 1
F1 = {item: count for item, count in item_counts.items() if count >= supMin}

# Armazena todos os Fk
all_frequents = F1.copy()
Fk = F1
k = 1

while Fk:
    k += 1
    # Etapa 5: gerar candidatos Ck a partir de Fk-1
    prev_items = list(Fk.keys())
    Ck = set()
    for i in range(len(prev_items)):
        for j in range(i + 1, len(prev_items)):
            union = tuple(sorted(set(prev_items[i]).union(set(prev_items[j]))))
            if len(union) == k:
                # Verificar que todos os subconjuntos (k-1) estão em Fk-1
                subsets = combinations(union, k - 1)
                if all(tuple(sorted(s)) in Fk for s in subsets):
                    Ck.add(union)

    # Etapa 6-7: contar suportes de Ck
    Ck_counts = {c: 0 for c in Ck}
    for t in transactions:
        t_set = set(t)
        for c in Ck:
            if set(c).issubset(t_set):
                Ck_counts[c] += 1

    # Etapa 8: gerar Fk
    Fk = {item: count for item, count in Ck_counts.items() if count >= supMin}
    all_frequents.update(Fk)

# Resultado
df_result = pd.DataFrame([
    {'itemsets': set(itemset), 'support': count / n_trans, 'length': len(itemset)}
    for itemset, count in all_frequents.items()
])

# Ordenar por tamanho e suporte
df_result = df_result.sort_values(by=['length', 'support'], ascending=[True, False]).reset_index(drop=True)

df_result

,itemsets,support,length
0,{I2},0.777778,1
1,{I1},0.666667,1
2,{I3},0.666667,1
3,{I5},0.222222,1
4,{I4},0.222222,1
5,"{I1, I3}",0.444444,2
6,"{I1, I2}",0.444444,2
7,"{I3, I2}",0.444444,2
8,"{I1, I5}",0.222222,2
9,"{I4, I2}",0.222222,2


# Usando o pacote mlxtend

In [32]:
# Transformar transações em formato one-hot
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

df

,I1,I2,I3,I4,I5
0,True,True,False,False,True
1,False,True,False,True,False
2,False,True,True,False,False
3,True,True,False,True,False
4,True,False,True,False,False
5,False,True,True,False,False
6,True,False,True,False,False
7,True,True,True,False,True
8,True,True,True,False,False


In [33]:
# Executar Apriori com suporte mínimo = 2 (~0.22)
frequent_itemsets = apriori(df, min_support=2/len(df), use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.666667,(I1)
1,0.777778,(I2)
2,0.666667,(I3)
3,0.222222,(I4)
4,0.222222,(I5)
5,0.444444,"(I1, I2)"
6,0.444444,"(I1, I3)"
7,0.222222,"(I1, I5)"
8,0.444444,"(I3, I2)"
9,0.222222,"(I4, I2)"


In [34]:
# Ordenar por tamanho dos conjuntos
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets.sort_values(by=['length', 'support'], ascending=[True, False])

frequent_itemsets

,support,itemsets,length
1,0.777778,(I2),1
0,0.666667,(I1),1
2,0.666667,(I3),1
3,0.222222,(I4),1
4,0.222222,(I5),1
5,0.444444,"(I1, I2)",2
6,0.444444,"(I1, I3)",2
8,0.444444,"(I3, I2)",2
7,0.222222,"(I1, I5)",2
9,0.222222,"(I4, I2)",2
